In [1]:
from random import choices, randint, randrange, random
from typing import List, Optional, Callable, Tuple
import numpy as np
import pandas as pd
import sobol_seq
import matplotlib.pyplot as plt
from functools import partial


In [2]:
Genome = List[int]
Population = List[Genome]
FitnessFunc = Callable[[Genome], float]
PopulateFunc = Callable[[], Population]
SelectionFunc = Callable[[Population, FitnessFunc], Tuple[Genome, Genome]]
CrossoverFunc = Callable[[Genome, Genome], Tuple[Genome, Genome]]
MutationFunc = Callable[[Genome], Genome]
PrinterFunc = Callable[[Population, int, FitnessFunc], None]

Schewel

In [3]:
# n_point = 100
# k_max = 100
# dim = 3
# epsilon = 10**(-7)
# num_bits = 64
# def objective_function(x):
#     f=0
#     for i in range (len(x)):
#         sum_sq = 0
#         for j in range (i+1):
#             sum_sq += x[j]
#         f += sum_sq**2 
#     return f
# boundaries = np.array([(-5,5) for _ in range (dim)])

2n Minima - Error

In [4]:
# n_point = 100
# k_max = 100
# dim = 3
# epsilon = 10**(-7)
# num_bits = 64
# def objective_function(x):
#     f=0
#     for i in range (len(x)):
#         f += (x[i]**4-16*x[i]**2+5*x[i])
#     return f
# boundaries = np.array([(-5,5) for _ in range (dim)])

Rastrigin

In [5]:
# n_point = 100
# k_max = 100
# dim = 3
# epsilon = 10**(-7)
# num_bits = 64
# def objective_function(x):
#     f=0
#     for i in range (len(x)):
#         f += (x[i]**2-10*np.cos(2*np.pi*x[i])+10)
#     return f
# boundaries = np.array([(-5,5) for _ in range (dim)])

Griewank

In [6]:
# n_point = 100
# k_max = 100
# dim = 3
# epsilon = 10**(-7)
# num_bits = 64
# def objective_function(x):
#     f=0
#     for i in range (len(x)):
#         f += x[i]**2
#     f = f/4000
#     for i in range (len(x)):
#         f *= np.cos(x[i]/np.sqrt(i))
#     return f
# boundaries = np.array([(-50,50) for _ in range (dim)])

Problem 1 - Paper Pak Kun

In [7]:
n_point = 100
k_max = 250
dim = 2
epsilon = 10**(-7)
num_bits = 64
p_mutation = 0.1

def objective_function(x):
    f1 = np.exp(x[0]-x[1])-np.sin(x[0]+x[1])
    f2 = (x[0]*x[1])**2-np.cos(x[0]+x[1])
    f_list = [f1,f2]
    denom = 0
    for f in f_list:
        denom +=np.abs(f)
    F = 1/(1+denom)
    return F
boundaries = np.array([(-10,10) for _ in range (dim)])

Problem One Equation

In [8]:
# n_point = 100
# k_max = 300
# dim = 1
# epsilon = 10**(-5)
# num_bits = 64
# p_mutation = 0.1

# def objective_function(x):
#     F = 1 / (1 + abs((x - 1) * (x + 2) * (x - 3)))
#     return F

# boundaries = np.array([(-10,10) for _ in range (dim)])

Problem 1 - Paper Mastorakis

In [9]:
# n_point = 100
# k_max = 300
# dim = 2
# epsilon = 10**(-5)
# num_bits = 64
# p_mutation = 0.1

# def objective_function(x):
#     F = (x[0]**2+x[0]*x[1]-6)**2+(x[0]**2+x[1]**3+2*x[0]*x[1]**2-3)**2
#     # F = abs(x[0]**2+x[0]*x[1]-6)+abs(x[0]**2+x[1]**3+2*x[0]*x[1]**2-3)
#     # F = x[0]**2+x[1]**2
#     return F
# boundaries = np.array([(-10,10) for _ in range (dim)])

Saddle Note Bifurcation

In [10]:
# n_point = 20
# k_max = 250
# dim = 2
# epsilon = 10**(-7)
# num_bits = 64

# def objective_function(X):
#     f1 = X[0]+X[1]**2 #-2*x
#     f_list = [f1]
#     denom = 1
#     for f in f_list:
#         denom +=np.abs(f)
#     F = 1/denom
#     return F

# boundaries = np.array([(-10,10) for _ in range (dim)])

In [11]:
min_value = boundaries.min()
max_value = boundaries.max()
# num_bits = 64  # Number of bits for each number
print(min_value,max_value)

-10 10


In [12]:
"""GENERATE POINTS USING SOBOL SEQUENCE"""
def generate_points(dim,npoint,low=-10,high=10):
    if type(low) != type(high):
        raise TypeError('The type of "low" and "high" should be the same.')
    if type(low) == int:
        boundaries = [(low,high) for _ in range (dim)]
    elif type(low) == list or type(low) == np.ndarray:
        if len(low) != len(high):
            raise TypeError('The length of "low" and "high" should be the same.')
        else:
            boundaries = [(low[i],high[i]) for i in range (len(low))]

    # Generate Sobol sequence points
    sobol_points = sobol_seq.i4_sobol_generate(dim, npoint)

    # Scale the Sobol points to fit within the specified boundaries
    scaled_points = []
    for i in range(dim):
        a, b = boundaries[i]
        scaled_dim = a + sobol_points[:, i] * (b - a)
        scaled_points.append(scaled_dim)

    # Transpose the scaled points to get points per dimension
    scaled_points = np.array(list(map(list, zip(*scaled_points))))
    return scaled_points

In [13]:
iter_points = generate_points(dim,n_point,boundaries[:,0],boundaries[:,1])
iter_points

array([[ 0.     ,  0.     ],
       [ 5.     , -5.     ],
       [-5.     ,  5.     ],
       [-2.5    , -2.5    ],
       [ 7.5    ,  7.5    ],
       [ 2.5    , -7.5    ],
       [-7.5    ,  2.5    ],
       [-6.25   , -3.75   ],
       [ 3.75   ,  6.25   ],
       [ 8.75   , -8.75   ],
       [-1.25   ,  1.25   ],
       [-3.75   , -6.25   ],
       [ 6.25   ,  3.75   ],
       [ 1.25   , -1.25   ],
       [-8.75   ,  8.75   ],
       [-8.125  , -0.625  ],
       [ 1.875  ,  9.375  ],
       [ 6.875  , -5.625  ],
       [-3.125  ,  4.375  ],
       [-0.625  , -8.125  ],
       [ 9.375  ,  1.875  ],
       [ 4.375  , -3.125  ],
       [-5.625  ,  6.875  ],
       [-6.875  , -6.875  ],
       [ 3.125  ,  3.125  ],
       [ 8.125  , -1.875  ],
       [-1.875  ,  8.125  ],
       [-4.375  , -4.375  ],
       [ 5.625  ,  5.625  ],
       [ 0.625  , -9.375  ],
       [-9.375  ,  0.625  ],
       [-9.0625 , -4.6875 ],
       [ 0.9375 ,  5.3125 ],
       [ 5.9375 , -9.6875 ],
       [-4.062

In [14]:
def encode_number(number, min_value=-10, max_value=10, num_bits=32):
    # Normalize the number to a value between 0 and 1
    normalized = (number - min_value) / (max_value - min_value)
    # Convert it to an integer representation
    int_representation = int(normalized * (2**num_bits - 1))
    # Convert the integer to binary and pad with zeros
    return [int(x) for x in format(int_representation, '0{}b'.format(num_bits))]

def decode_number(genome:Genome, min_value=-10, max_value=10, num_bits=32):
    # Convert the binary string to an integer
    int_representation = int(''.join(map(str, genome)), 2)
    # Scale down to the normalized value
    normalized = int_representation / (2**num_bits - 1)
    # Denormalize to get the real number
    return min_value + normalized * (max_value - min_value)

def encode_list(number_list,min_value=-10, max_value=10, num_bits=32):
    encoded = []
    for number in number_list:
        encoded += encode_number(number,min_value, max_value, num_bits)
    return encoded

def decode_list(encoded_list,min_value=-10, max_value=10, num_bits=32):
    numbers = []
    for i in range(0, len(encoded_list), num_bits):
        binary_list = encoded_list[i:i + num_bits]
        number = decode_number(binary_list, min_value, max_value, num_bits)
        numbers.append(number)
    return numbers


In [15]:
def generate_population(set_of_points:np.ndarray,min_value=-10, max_value=10, num_bits=32) -> Population:
    return [encode_list(set_of_points[point],min_value, max_value, num_bits) for point in range(len(set_of_points))]

def single_point_crossover(a: Genome, b: Genome, print_cutoff=False) -> Tuple[Genome, Genome]:
    if len(a) != len(b):
        raise ValueError("Genomes a and b must be of same length")
    length = len(a)
    # if the length less than 2, then there is no point to do the function
    if length < 2:  
        return a, b
    # generate random number as the cutoff of the crossover
    p = randint(1, length - 1)
    if print_cutoff == True:
        print(p)
    
    return a[0:p] + b[p:], b[0:p] + a[p:]

def mutation(genome: Genome, num: int = 1, probability: float = 0.5) -> Genome:
    # num: generate how many chromosome(s) that we want to mutate
    for _ in range(num):
        # index sets which chromosome we want to change
        index = randrange(len(genome))
        # the change algorithm
        genome[index] = genome[index] if random() > probability else abs(genome[index] - 1)
    return genome

def fitness_function(genome: Genome,objective_function, min_value=-10, max_value=10, num_bits=64) -> float:
    X = decode_list(genome, min_value, max_value, num_bits)
    return objective_function(X)

# for convenience, call fitness function from these functions below using partial(...)
def population_fitness(population: Population, fitness_func: FitnessFunc) -> float:
    return sum([fitness_func(genome) for genome in population])

def selection_pair(population: Population,population_fitness_func, fitness_func: FitnessFunc,minimize=True) -> Population:
    total_sum = population_fitness_func(population,fitness_func)
    if minimize == True:
        return choices(
            population=population,
            weights=[(total_sum-fitness_func(gene))/total_sum for gene in population],
            k=2
        )
    else:
            return choices(
            population=population,
            weights=[fitness_func(gene) for gene in population],
            k=2
        )
def sort_population(population: Population, fitness_func: FitnessFunc, minimize=True) -> Population:
    return sorted(population,key= lambda x: fitness_func(x), reverse=not minimize)

def genome_to_string(genome: Genome) -> str:
    return "".join(map(str, genome))

def print_stats(population: Population, generation_id: int, fitness_func: FitnessFunc, decode_list_func,binary_mode=False):
    print("GENERATION %02d" % generation_id)
    print("=============")
    if binary_mode == True:
        print("Population: [%s]" % ", ".join([genome_to_string(gene) for gene in population]))
        print("Avg. Fitness: %f" % (population_fitness(population, fitness_func) / len(population)))
        print("Worst solution: %s (%f)" % (genome_to_string(population[-1]),
                                fitness_func(population[-1])))
        print(
            "Best solution: %s (%f)" % (genome_to_string(population[0]), fitness_func(population[0])))
        print("")
    else:
        print(f"Population: {[decode_list_func(population[i]) for i in range (len(population))]}")
        print("Avg. Fitness: %f" % (population_fitness(population, fitness_func) / len(population)))
        print(f"Worst solution: {(decode_list_func(population[-1]))} with value {fitness_func(population[-1])}")
        print(f"Best solution: {(decode_list_func(population[0]))} with value {fitness_func(population[0])}")
        print("")
    # return sorted_population[0]

def run_evolution(
        populate_func,
        fitness_func,
        fitness_limit: int,
        minimize = False,
        selection_func: SelectionFunc = selection_pair,
        crossover_func: CrossoverFunc = single_point_crossover,
        mutation_func: MutationFunc = mutation,
        sort_func = sort_population,
        generation_limit: int = 100,
        printer: Optional[PrinterFunc] = None) \
        -> Tuple[Population, int]:
    population = populate_func()

    for i in range(generation_limit):
        # print(f"i={i}")
        population = sort_func(population,minimize=minimize)

        if printer is not None:
            printer(population, i)

        if minimize==True:
            cutoff_criteria = fitness_func(population[0])
        else:
            cutoff_criteria = 1-fitness_func(population[0])
        print(f"cutoff:{cutoff_criteria}\n")
        # print(decode_list(population[0],min_value=min_value,max_value=max_value,num_bits=64))
        if cutoff_criteria < fitness_limit:
            break

        next_generation = population[0:2]

        for j in range(int(len(population) / 2) - 1):
            parents = selection_func(population,minimize=minimize)
            # print(f"j={j}")
            # print(decode_list(parents[0],min_value=min_value,max_value=max_value,num_bits=64),decode_list(parents[1],min_value=min_value,max_value=max_value,num_bits=64))
            offspring_a, offspring_b = crossover_func(parents[0], parents[1])
            offspring_a = mutation_func(offspring_a)
            offspring_b = mutation_func(offspring_b)
            next_generation += [offspring_a, offspring_b]

        population = next_generation

    return population, i

In [16]:
population,generation = run_evolution(
    populate_func=partial(
        generate_population,set_of_points = iter_points,num_bits=num_bits,min_value=min_value,max_value=max_value
    ),
    fitness_func= partial(
        fitness_function,objective_function=objective_function, num_bits=num_bits,min_value=min_value,max_value=max_value
    ),
    minimize=False,
    sort_func=partial(
        sort_population,fitness_func=partial(
            fitness_function,
            objective_function=objective_function,
            num_bits=num_bits,
            min_value=min_value,max_value=max_value   
        )),
    selection_func=partial(
        selection_pair,
        population_fitness_func=population_fitness,
        fitness_func=partial(
            fitness_function,
            objective_function=objective_function,
            num_bits=num_bits,
            min_value=min_value,max_value=max_value)),
    mutation_func=partial(
        mutation,probability=p_mutation
    ),
    fitness_limit=epsilon,
    generation_limit=k_max,
    printer=partial(print_stats,fitness_func=partial(
        fitness_function,objective_function=objective_function,max_value=max_value,min_value=min_value,num_bits=num_bits
        ),decode_list_func=partial(
            decode_list,min_value=min_value,max_value=max_value,num_bits=num_bits)
        )
)

GENERATION 00
Population: [[-1.25, 1.25], [-1.40625, 1.09375], [0.0, 0.0], [-0.15625, -0.15625], [-4.0625, 0.3125], [-0.3125, 6.5625], [0.78125, 2.65625], [-2.8125, -0.9375], [1.25, -1.25], [-0.9375, -2.8125], [1.09375, -1.40625], [0.46875, 7.96875], [0.9375, 5.3125], [-8.125, -0.625], [2.34375, 2.34375], [-9.375, 0.625], [-2.5, -2.5], [1.5625, 4.6875], [-0.78125, 9.21875], [-2.1875, 3.4375], [4.6875, 1.5625], [0.3125, -4.0625], [-1.71875, -4.84375], [3.125, 3.125], [5.3125, 0.9375], [-3.28125, -3.28125], [-7.1875, -1.5625], [-5.9375, 2.1875], [-4.53125, 2.96875], [-3.125, 4.375], [-8.28125, 1.71875], [-1.875, 8.125], [1.875, 9.375], [4.21875, 4.21875], [3.4375, -2.1875], [-7.5, 2.5], [-4.375, -4.375], [-9.53125, -2.03125], [-2.65625, 7.34375], [-1.5625, -7.1875], [3.59375, 6.09375], [6.09375, 3.59375], [-6.25, -3.75], [3.75, 6.25], [-3.75, -6.25], [6.25, 3.75], [2.8125, 8.4375], [-5.15625, 4.84375], [-5.0, 5.0], [-6.40625, -3.90625], [-3.90625, -6.40625], [8.4375, 2.8125], [6.5625, -0

GENERATION 01
Population: [[-0.15625, 6.5625], [-0.78125, 1.09375], [0.0, 1.25], [0.46875, 1.25], [-0.3125, 1.25], [-0.15625, 1.25], [-0.15625, 1.25], [-0.15625, 1.25], [-0.1556396484375, 1.09375], [-1.25, 1.09375], [-1.25, 1.09375], [0.78125, 1.09375], [-1.40625, -0.15625], [-1.25, -0.15625], [-1.25, -0.15625], [-1.25, -0.15625], [-1.25, 0.3125], [-1.25, 0.3125], [-1.25, 1.25], [-1.25, 1.25], [-1.25, 1.25], [-1.25, 1.25], [-1.25, 1.25], [-1.25, 1.25], [-1.25, 1.25], [-1.25, 1.25], [-1.25, 0.0], [-4.0625, -0.15625], [-1.406249999990905, 1.09375], [-1.40625, 1.09375], [-1.40625, 1.09375], [-1.40625, 1.09375], [-1.40625, 1.09375], [-1.40625, 1.09375], [-1.40625, 1.09375], [-1.40625, 1.09375], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [-2.5, 0.3125], [-0.3125, -0.15625], [-0.15625, -0.15625], [-0.15625, -0.15625], [-0.15625, -0.15625], [-0.15625, -0.15625], [-0.15625, -0.15625], [-0.15625, -0.15625], [-0.15625, -0.15625], [-0.1556396484375, -0.15625], [-1.40625, 1.25], [-1.40625, 1.25], [1.0937

In [17]:
popA = generate_population(iter_points,min_value=min_value,max_value=max_value,num_bits=num_bits)

for i in range(k_max):
    # print(f"i={i}")
    population = sort_func(population,minimize=minimize)

    if printer is not None:
        printer(population, i)

    if minimize==True:
        cutoff_criteria = fitness_func(population[0])
    else:
        cutoff_criteria = 1-fitness_func(population[0])
    print(f"cutoff:{cutoff_criteria}\n")
    # print(decode_list(population[0],min_value=min_value,max_value=max_value,num_bits=64))
    if cutoff_criteria < fitness_limit:
        break

    next_generation = population[0:2]

    for j in range(int(len(population) / 2) - 1):
        parents = selection_func(population,minimize=minimize)
        # print(f"j={j}")
        # print(decode_list(parents[0],min_value=min_value,max_value=max_value,num_bits=64),decode_list(parents[1],min_value=min_value,max_value=max_value,num_bits=64))
        offspring_a, offspring_b = crossover_func(parents[0], parents[1])
        offspring_a = mutation_func(offspring_a)
        offspring_b = mutation_func(offspring_b)
        next_generation += [offspring_a, offspring_b]

    population = next_generation


NameError: name 'sort_func' is not defined

In [ ]:
new_population = []
for i in range (len(population)):
    print(decode_list(population[i],min_value=min_value,max_value=max_value,num_bits=num_bits))
    print(objective_function(decode_list(population[i],min_value=min_value,max_value=max_value,num_bits=num_bits)))

[-3.125, 1.171875]
0.05402876915468369
[-3.125, 1.171875]
0.05402876915468369
[-2.8124999999249667, 1.1718750000022737]
3.372982022833404
[-3.1249999999249667, 1.1718750000024158]
0.05402876919424032
[-3.125, 1.1718750000024158]
0.05402876916370796
[-3.1249999999977263, 0.859375]
8.920073418107163
[-3.124980926513672, 1.19140625]
0.17390120186061003
[-3.125, 1.171875]
0.05402876915468369
[-2.8124999999977263, 1.1914062500000018]
3.988515848161251
[-3.125, 1.1718751024460516]
0.05402915183336629
[-3.125, 1.171875]
0.05402876915468369
[-3.125, 1.1718751024454832]
0.054029151831243044
[-3.124847412109375, 1.171875]
0.054091741652171454
[-3.1249999999977263, 1.1718750000000355]
0.05402876915574166
[-2.8124999999977263, 1.1718750005821121]
3.3729820386189804
[-3.124999999999999, 1.1718751024454832]
0.05402915183124287
[-2.8124999999249667, 1.171875]
3.372982022766001
[-3.124999999993179, 1.1718750000022737]
0.054028769165952936
[-3.124999999990905, 1.171875]
0.05402876915838498
[-3.12484741

In [ ]:
new_population = []
for i in range (len(population)):
    new_population.append(decode_list(population[i],min_value=min_value,max_value=max_value,num_bits=num_bits))
new_population

[[-3.125, 1.171875],
 [-3.125, 1.171875],
 [-2.8124999999249667, 1.1718750000022737],
 [-3.1249999999249667, 1.1718750000024158],
 [-3.125, 1.1718750000024158],
 [-3.1249999999977263, 0.859375],
 [-3.124980926513672, 1.19140625],
 [-3.125, 1.171875],
 [-2.8124999999977263, 1.1914062500000018],
 [-3.125, 1.1718751024460516],
 [-3.125, 1.171875],
 [-3.125, 1.1718751024454832],
 [-3.124847412109375, 1.171875],
 [-3.1249999999977263, 1.1718750000000355],
 [-2.8124999999977263, 1.1718750005821121],
 [-3.124999999999999, 1.1718751024454832],
 [-2.8124999999249667, 1.171875],
 [-3.124999999993179, 1.1718750000022737],
 [-3.124999999990905, 1.171875],
 [-3.1248474121048275, 0.8593761926750076],
 [-3.0859375, 1.171875],
 [-3.125, 1.171875],
 [-3.125, 1.171875],
 [-3.125, 1.171875],
 [-2.812499999993179, 1.1718750005821121],
 [-3.124980926513671, 1.1718750000727596],
 [-3.124847412109374, 1.1719512939453125],
 [-3.125, 1.171875],
 [-3.125, 1.1718751024454832],
 [-3.125, 1.1718761920928955],
 [-3

In [ ]:
import random

def multi_point_crossover(parent1, parent2, parent3, parent4, num_points):
    length = len(parent1)
    points = sorted(random.sample(range(1, length), num_points))
    offspring = []
    parents = [parent1, parent2, parent3, parent4]

    previous_point = 0
    for i, point in enumerate(points + [length]):
        offspring.extend(parents[i % 4][previous_point:point])
        previous_point = point

    return offspring

# Example usage
parent1 = [1, 1, 1, 1, 1, 1, 1, 1]
parent2 = [2, 2, 2, 2, 2, 2, 2, 2]
parent3 = [3, 3, 3, 3, 3, 3, 3, 3]
parent4 = [4, 4, 4, 4, 4, 4, 4, 4]

offspring = multi_point_crossover(parent1, parent2, parent1, parent2, 2)
print(offspring)


[1, 2, 2, 2, 2, 2, 2, 1]


In [ ]:
def single_point_crossover(parent1: Genome, parent2: Genome, print_cutoff=False) -> Tuple[Genome, Genome]:
    if len(parent1) != len(parent2):
        raise ValueError("Genomes a and b must be of same length")
    length = len(parent1)
    # if the length less than 2, then there is no point to do the function
    if length < 2:  
        return parent1, parent2
    # generate random number as the cutoff of the crossover
    p = randint(1, length - 1)
    if print_cutoff == True:
        print(p)
    
    return parent1[0:p] + parent2[p:], parent2[0:p] + parent1[p:]
single_point_crossover(parent1,parent2)

([1, 1, 1, 1, 2, 2, 2, 2], [2, 2, 2, 2, 1, 1, 1, 1])

In [ ]:
import random

def crossover_4k_genes(parent1, parent2):
    length = len(parent1)
    children = []

    # Define crossover points
    crossover_points = sorted(random.sample(range(1, length), 2))

    # Create four offspring using different combinations of parent segments
    for i in range(2):
        child1 = parent1[:crossover_points[i]] + parent2[crossover_points[i]:]
        child2 = parent2[:crossover_points[i]] + parent1[crossover_points[i]:]
        children.extend([child1, child2])

    return children

crossover_4k_genes(parent1,parent2)

[[1, 1, 1, 1, 2, 2, 2, 2],
 [2, 2, 2, 2, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 2, 2],
 [2, 2, 2, 2, 2, 2, 1, 1]]

In [ ]:
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.problems import get_problem
from pymoo.optimize import minimize

problem = get_problem("g1")

algorithm = GA(
    pop_size=100,
    eliminate_duplicates=True)

res = minimize(problem,
               algorithm,
               seed=1,
               verbose=False)

print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

Best solution found: 
X = [1.         1.         1.         1.         1.         1.
 1.         1.         1.         2.99994548 2.98724675 2.99782144
 1.        ]
F = [-14.98501368]
